In [1]:
# Scrape KenPom
# https://kenpom.com/index.php?y=2008
# https://www.kaggle.com/code/walterhan/scrape-kenpom-data
# https://www.geeksforgeeks.org/scrape-tables-from-any-website-using-python/
# https://medium.com/analytics-vidhya/how-to-scrape-a-table-from-website-using-python-ce90d0cfb607


This is typing space.

In [2]:
# Grab KenPom 2008 to 2023

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

season = np.arange(2008,2024,1)
allkp = pd.DataFrame(columns = ['Year','Team','Conf','AdjEM','AdjO','AdjD','AdjT','SOSEM','SOSO','SOSD'])
for y in season:
    req = Request(
        url=f'https://kenpom.com/index.php?y={y}', 
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    webpage = urlopen(req).read()
    soup =BeautifulSoup(webpage, 'html.parser')

    table = soup.find('table', class_=None)
    
    for row in table.tbody.find_all('tr'):    
        # Find all data for each column
        columns = row.find_all('td')
        
        if(columns != []):
            
            Team = columns[1].text.strip()
            Conf = columns[2].text.strip()
            
            AdjEM = columns[4].text.strip()
            AdjO = columns[5].text.strip()
            AdjD = columns[7].text.strip()
            AdjT = columns[9].text.strip()
            SOSEM = columns[13].text.strip()
            SOSO = columns[15].text.strip()
            SOSD = columns[17].text.strip()
            newrow = [y,Team,Conf,AdjEM,AdjO,AdjD,AdjT,SOSEM,SOSO,SOSD]
            allkp.loc[len(allkp)]=newrow
            #newrow = pd.DataFrame({'Year':2008,'Team': Team,'Conf':Conf,'AdjEM':AdjEM,'AdjO':AdjO,'AdjD':AdjD,'AdjT':AdjT,'SOSEM':SOSEM,'SOSO':SOSO,'SOSD':SOSD})
            #allkp = pd.concat(allkp,newrow)
            allkp['Team'] = allkp['Team'].str.replace('\d+','',regex=True).str.replace('*','',regex=True).str.rstrip()
allkp.to_csv('step01_kenpom0824.csv',index=False)


In [21]:
# Grab KenPom 2024

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


allkp = pd.read_csv('step01_kenpom0824.csv')
allkp = allkp[allkp['Year']<2024]
y = 2024

req = Request(
    url=f'https://kenpom.com/index.php?y={y}', 
    headers={'User-Agent': 'Mozilla/5.0'}
)
webpage = urlopen(req).read()
soup =BeautifulSoup(webpage, 'html.parser')

table = soup.find('table', class_=None)



In [22]:


for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        
        Team = columns[1].text.strip()
        Conf = columns[2].text.strip()
        
        AdjEM = columns[4].text.strip()
        AdjO = columns[5].text.strip()
        AdjD = columns[7].text.strip()
        AdjT = columns[9].text.strip()
        SOSEM = columns[13].text.strip()
        SOSO = columns[15].text.strip()
        SOSD = columns[17].text.strip()
        newrow = [y,Team,Conf,AdjEM,AdjO,AdjD,AdjT,SOSEM,SOSO,SOSD]
        allkp.loc[len(allkp)]=newrow
        #newrow = pd.DataFrame({'Year':2008,'Team': Team,'Conf':Conf,'AdjEM':AdjEM,'AdjO':AdjO,'AdjD':AdjD,'AdjT':AdjT,'SOSEM':SOSEM,'SOSO':SOSO,'SOSD':SOSD})
        #allkp = pd.concat(allkp,newrow)
        #allkp['Team'] = allkp['Team'].str.replace('\d+','',regex=True).str.replace('*','',regex=True).str.rstrip()
        def remove_trailing_space_or_digit(text):
 
            while text and (text[-1].isspace() or text[-1].isdigit()):
                text = text[:-1]
            return text

        # Apply the lambda function to the 'A' column
        allkp['Team'] = allkp['Team'].apply(remove_trailing_space_or_digit)




In [23]:
allkp.to_csv('step01_kenpom0824.csv',index=False)
a = allkp['Team'].unique()
b = pd.DataFrame({'tm':sorted(a)})
b.to_csv('aaa.csv',index=False)